# K-Nearest Neighbors Algorithim:  Python Class Implementation


In [1]:
# import libraries
import requests
from contextlib import closing
import csv
import pandas as pd
from random import seed
from sklearn.preprocessing import MinMaxScaler



In [9]:
cols = ['sepal_length', ' sepal_width', 'petal_length', 'petal_width', 'class']
df = pd.read_csv('IRIS.csv', names=cols, skiprows=1)
df.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [10]:
df.describe()


,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1    sepal_width  150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   class         150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [12]:
def load_csv(file_name):
    """
    load CSV file into pandas dataframe
    """
    
    dataframe = pd.read_csv(file_name, names=cols)

        
    return dataframe


In [13]:
# test load_csv()
file_name = 'IRIS.csv'
dataframe = load_csv(file_name)

print(dataframe.head())


   sepal_length  sepal_width  petal_length  petal_width        class
0  sepal_length  sepal_width  petal_length  petal_width      species
1           5.1          3.5           1.4          0.2  Iris-setosa
2           4.9            3           1.4          0.2  Iris-setosa
3           4.7          3.2           1.3          0.2  Iris-setosa
4           4.6          3.1           1.5          0.2  Iris-setosa


In [23]:
# Data Wrangling
def string_to_float(dataframe, col):
    """
    convert string column values to floats;
    dataframe showing dtype of objects -> 
    assuming mixed dtypes of strings, ints and floats   
    """
#    try:
#    s = pd.Series(dataframe[col])
    # remove white space at end and beginning of string text
    # return a floating point number from a number or a string
    # dataframe = dataframe.replace('^\s+', '', regex=True, inplace=True)
    dataframe = dataframe.applymap(lambda x: x.str.strip() if x.dtype == "str" else x)
    dataframe[col] = dataframe[col].apply(lambda x: x.astype(float) if x.dtype == "str" else x)
    # dataframe[col] = pd.to_numeric(dataframe[col], errors='coerce')
#     s = s.str.strip()
#     s = s.astype(float)
#     dataframe.append(s)
        
#     except ValueError:
#         print("Oops! Please enter a column containing numbers!")
  
def ints_to_floats(dataframe, col):
    """
    convert integers values to floats;
    dataframe showing dtype of objects -> 
    assuming mixed dtypes of strings, ints and floats   
    """
    pass
    


In [24]:
print(dataframe.head())

dataframe.info()
# test data wrangling 
string_to_float(dataframe, "sepal_length")

    

   sepal_length  sepal_width  petal_length  petal_width        class
0  sepal_length  sepal_width  petal_length  petal_width      species
1           5.1          3.5           1.4          0.2  Iris-setosa
2           4.9            3           1.4          0.2  Iris-setosa
3           4.7          3.2           1.3          0.2  Iris-setosa
4           4.6          3.1           1.5          0.2  Iris-setosa
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   sepal_length  151 non-null    object
 1    sepal_width  151 non-null    object
 2   petal_length  151 non-null    object
 3   petal_width   151 non-null    object
 4   class         151 non-null    object
dtypes: object(5)
memory usage: 6.0+ KB


AttributeError: 'str' object has no attribute 'dtype'

In [ ]:
# Pre-Processing
def normalize():
    """
    rescale dataset columns to range 0-1;
    must be used after train/test split to avoid leakage/bias
    """
    data = [X_train]
    scaler = MinMaxScaler()
    
    scaler.fit.transform(data)
    
    scaler.
    pass

def create_unique_class_id(dataframe, col):
    """
    identify unique classification values; 
    convert column values to integers
    """
    s = pd.Series(dataframe[col])
    class_val_labels, levels = pd.factorize(s)
        
    return class_val_labels

def calculate_ed():
    """
    calculate euclidean distance between two vectors
    """
    pass

# def min_max(dataframe):
#     """
#     find the minimum and maximum values for each column;
#     create a list of these values
#     """
#     # create empty list
#     minmax = []
        
#     # groups the data by columns and assigns the value of 
#     # the minimum value per group to the min variable
#     min = dataframe.groupby(col).min()

#     # groups the data by columns and assigns the value of
#     # the maximum value per group to the max variable
#     max = dataframe.groupby(col).max()
    
#     for i in range(len(dataframe[0])):
#         minmax.append([min, max])
        
#     return minmax

## Note: can use MinMaxScaler fit to accomplish this and then transform to scale for normalization



# def mean():
#     """
#     groups the data by columns and returns the mean value per group
#     """
#     return dataframe.groupby(col).mean()


In [ ]:
# test preprocessing


In [ ]:
# Implement K-NN Algorithm
def get_neighbors():
    """
    locate most similar neighbors
    """
    pass

def predict_class():
    """
    make a prediction with neighbors
    """
    pass

def k_nn():
    """
    K-NN algorithm
    """
    pass


In [ ]:
# test implementation


In [ ]:
# Evaluate K-NN Algorithm
def cross_val_split():
    """
    split dataset into k folds
    """
    pass

def accuracy_metric():
    """
    calculate accuracy percentage
    """
    pass

def cross_val_metric():
    """
    evaluate algorithm using cross validation 
    """
    pass


In [ ]:
# test evaluation


In [ ]:
# Refactored (complete) Algorithm

# class KNN:
#     def __init__(self, name):
#         self.name = name


In [ ]:
# Test refactored code
